# Election Data Silver Layer Processing

This notebook processes the Bronze layer election data and creates a Silver layer with:
- Data cleaning and standardization
- Data type corrections
- Data validation and quality improvements
- Basic transformations and enrichments

In [12]:
# Install required packages
!pip install pyspark findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
import re
from datetime import datetime

In [13]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Election Silver Layer Processing") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

print("🚀 Spark Session initialized for Silver Layer Processing!")
spark.sparkContext.setLogLevel("WARN")
spark

🚀 Spark Session initialized for Silver Layer Processing!


In [14]:
def load_bronze_data(bronze_path="/content/election_results"):
    """
    Load Bronze layer data from parquet files

    Args:
        bronze_path (str): Path to bronze layer parquet files

    Returns:
        pyspark.sql.DataFrame: Bronze layer DataFrame
    """
    try:
        # Try reading as CSV with header and schema inference
        bronze_df = spark.read.csv(bronze_path, header=True, inferSchema=True)
        print(f"✅ Successfully loaded Bronze data from {bronze_path} as CSV")
        print(f"📊 Total records: {bronze_df.count()}")
        print(f"📋 Columns: {len(bronze_df.columns)}")
        return bronze_df
    except Exception as e:
        print(f"❌ Error loading Bronze data: {e}")
        return None

In [15]:
def clean_and_standardize_data(df):
    """
    Clean and standardize the election data

    Args:
        df (pyspark.sql.DataFrame): Input Bronze DataFrame

    Returns:
        pyspark.sql.DataFrame: Cleaned DataFrame
    """
    print("🧹 Starting data cleaning and standardization...")

    # Clean and standardize string columns
    silver_df = df \
        .withColumn("State_Clean",
                   regexp_replace(trim(col("State")), r"\s+", " ")) \
        .withColumn("Constituency_Clean",
                   regexp_replace(trim(upper(col("Constituency"))), r"\s+", " ")) \
        .withColumn("Candidate_Clean",
                   regexp_replace(trim(upper(col("Candidate"))), r"\s+", " ")) \
        .withColumn("Party_Clean",
                   regexp_replace(trim(col("Party")), r"\s+", " "))

    # Handle EVM Votes - convert string to integer, handle '-' as 0
    silver_df = silver_df \
        .withColumn("EVM_Votes_Clean",
                   when(col("EVM_Votes") == "-", 0)
                   .otherwise(col("EVM_Votes").cast(IntegerType())))

    # Handle Postal Votes - convert string to integer, handle '-' as 0
    silver_df = silver_df \
        .withColumn("Postal_Votes_Clean",
                   when(col("Postal_Votes") == "-", 0)
                   .otherwise(col("Postal_Votes").cast(IntegerType())))

    # Clean percentage column - remove any non-numeric characters and convert to double
    silver_df = silver_df \
        .withColumn("Vote_Percentage",
                   regexp_replace(col("percent_of_Votes"), "[^0-9.]", "")
                   .cast(DoubleType()))

    # Standardize Result column
    silver_df = silver_df \
        .withColumn("Result_Status",
                   when(upper(col("Result")) == "WON", "Winner")
                   .otherwise("Runner-up"))

    print("✅ Data cleaning and standardization completed")
    return silver_df

In [16]:
def add_derived_columns(df):
    """
    Add derived columns and business logic

    Args:
        df (pyspark.sql.DataFrame): Input cleaned DataFrame

    Returns:
        pyspark.sql.DataFrame: DataFrame with derived columns
    """
    print("🔧 Adding derived columns...")

    # Calculate total votes (should match Total_Votes column)
    silver_df = df \
        .withColumn("Calculated_Total_Votes",
                   col("EVM_Votes_Clean") + col("Postal_Votes_Clean"))

    # Add vote margin calculation (for winners vs runner-ups)
    window_spec = Window.partitionBy("State_Clean", "Constituency_Clean").orderBy(desc("Total_Votes"))

    silver_df = silver_df \
        .withColumn("Rank_in_Constituency", row_number().over(window_spec)) \
        .withColumn("Is_Winner", when(col("Rank_in_Constituency") == 1, True).otherwise(False))

    # Add party type classification
    major_parties = ["Bharatiya Janata Party", "Indian National Congress",
                    "Samajwadi Party", "Bahujan Samaj Party", "All India Trinamool Congress",
                    "Dravida Munnetra Kazhagam", "All India Anna Dravida Munnetra Kazhagam",
                    "Communist Party of India (Marxist)", "Communist Party of India",
                    "Nationalist Congress Party", "Shiv Sena", "Janata Dal (United)",
                    "Rashtriya Janata Dal", "Biju Janata Dal", "Telugu Desam Party",
                    "YSR Congress Party", "Telangana Rashtra Samithi"]

    silver_df = silver_df \
        .withColumn("Party_Type",
                   when(col("Party_Clean") == "Independent", "Independent")
                   .when(col("Party_Clean") == "None of the Above", "NOTA")
                   .when(col("Party_Clean").isin(major_parties), "National/Regional")
                   .otherwise("Others"))

    # Add processing timestamp for Silver layer
    silver_df = silver_df \
        .withColumn("Silver_Processing_Time", current_timestamp())

    # Add vote share categories
    silver_df = silver_df \
        .withColumn("Vote_Share_Category",
                   when(col("Vote_Percentage") >= 50, "Majority (50%+)")
                   .when(col("Vote_Percentage") >= 25, "Strong (25-49%)")
                   .when(col("Vote_Percentage") >= 10, "Moderate (10-24%)")
                   .when(col("Vote_Percentage") >= 5, "Low (5-9%)")
                   .otherwise("Minimal (<5%)"))

    print("✅ Derived columns added successfully")
    return silver_df

In [17]:
def validate_silver_data(df):
    """
    Perform data validation and quality checks for Silver layer

    Args:
        df (pyspark.sql.DataFrame): Silver DataFrame to validate
    """
    print("🔍 Performing Silver layer data validation...")

    # Check for data consistency
    print("\n=== Data Consistency Checks ===")

    # Check if calculated votes match original total votes
    inconsistent_votes = df.filter(col("Calculated_Total_Votes") != col("Total_Votes")).count()
    print(f"Records with vote calculation mismatch: {inconsistent_votes}")

    # Check for negative values
    negative_evm = df.filter(col("EVM_Votes_Clean") < 0).count()
    negative_postal = df.filter(col("Postal_Votes_Clean") < 0).count()
    print(f"Records with negative EVM votes: {negative_evm}")
    print(f"Records with negative Postal votes: {negative_postal}")

    # Check percentage ranges
    invalid_percentage = df.filter((col("Vote_Percentage") < 0) | (col("Vote_Percentage") > 100)).count()
    print(f"Records with invalid vote percentage: {invalid_percentage}")

    # Check for exactly one winner per constituency
    print("\n=== Winner Validation ===")
    constituencies_with_multiple_winners = df.filter(col("Is_Winner") == True) \
        .groupBy("State_Clean", "Constituency_Clean").count() \
        .filter(col("count") > 1).count()
    print(f"Constituencies with multiple winners: {constituencies_with_multiple_winners}")

    constituencies_with_no_winners = df.groupBy("State_Clean", "Constituency_Clean") \
        .agg(sum(when(col("Is_Winner") == True, 1).otherwise(0)).alias("winner_count")) \
        .filter(col("winner_count") == 0).count()
    print(f"Constituencies with no winners: {constituencies_with_no_winners}")

    # Summary statistics
    print("\n=== Silver Layer Summary ===")
    print(f"Total records: {df.count()}")
    print(f"Total constituencies: {df.select('State_Clean', 'Constituency_Clean').distinct().count()}")
    print(f"Total states: {df.select('State_Clean').distinct().count()}")
    print(f"Total unique parties: {df.select('Party_Clean').distinct().count()}")

    # Party type distribution
    print("\n=== Party Type Distribution ===")
    df.groupBy("Party_Type").count().orderBy(desc("count")).show()

    # Vote share category distribution
    print("\n=== Vote Share Category Distribution ===")
    df.groupBy("Vote_Share_Category").count().orderBy(desc("count")).show()

    print("✅ Silver layer validation completed")

In [18]:
def select_silver_columns(df):
    """
    Select and organize final columns for Silver layer

    Args:
        df (pyspark.sql.DataFrame): Input DataFrame with all transformations

    Returns:
        pyspark.sql.DataFrame: Final Silver DataFrame
    """
    print("📋 Selecting final Silver layer columns...")

    silver_final = df.select(
        # Original identifiers
        col("ID").alias("Candidate_ID"),
        col("row_id").alias("Bronze_Row_ID"),

        # Cleaned geographic information
        col("State_Clean").alias("State"),
        col("Constituency_Clean").alias("Constituency"),

        # Candidate and party information
        col("Candidate_Clean").alias("Candidate"),
        col("Party_Clean").alias("Party"),
        col("Party_Type"),

        # Vote data (cleaned)
        col("EVM_Votes_Clean").alias("EVM_Votes"),
        col("Postal_Votes_Clean").alias("Postal_Votes"),
        col("Total_Votes"),
        col("Calculated_Total_Votes"),
        col("Vote_Percentage"),
        col("Vote_Share_Category"),

        # Result information
        col("Result_Status"),
        col("Is_Winner"),
        col("Rank_in_Constituency"),

        # Metadata
        col("IngestationTime").alias("Bronze_Ingestion_Time"),
        col("Silver_Processing_Time"),
        col("File_name").alias("Source_File")
    )

    print("✅ Final Silver layer columns selected")
    return silver_final

In [19]:
def save_silver_data(df, output_path="/content/silver_layer/election_results"):
    """
    Save Silver layer data to parquet format

    Args:
        df (pyspark.sql.DataFrame): Silver DataFrame to save
        output_path (str): Output path for parquet files
    """
    print(f"💾 Saving Silver layer data to {output_path}...")

    try:
        df.write \
            .mode("overwrite") \
            .option("compression", "snappy") \
            .csv(output_path)

        print(f"✅ Silver data successfully saved to: {output_path}")
        print(f"📊 Records saved: {df.count()}")

        # Also save as CSV for easy viewing
        csv_path = output_path.replace("silver_layer", "silver_layer_csv")
        df.coalesce(1).write \
            .mode("overwrite") \
            .option("header", "true") \
            .csv(csv_path)

        print(f"📄 Silver data also saved as CSV to: {csv_path}")

    except Exception as e:
        print(f"❌ Error saving Silver data: {e}")

In [20]:
def process_silver_layer(bronze_path="/content/election_results",
                        silver_path="/content/sample_data/election_results_silver"):
    """
    Main orchestration function for Silver layer processing

    Args:
        bronze_path (str): Path to Bronze layer data
        silver_path (str): Output path for Silver layer data

    Returns:
        pyspark.sql.DataFrame: Final Silver DataFrame
    """
    print("🏗️ Starting Silver Layer Processing Pipeline...")
    print("=" * 60)

    # Step 1: Load Bronze data
    bronze_df = load_bronze_data(bronze_path)
    if bronze_df is None:
        print("❌ Failed to load Bronze data. Exiting.")
        return None

    print("\n" + "=" * 60)

    # Step 2: Clean and standardize data
    cleaned_df = clean_and_standardize_data(bronze_df)

    print("\n" + "=" * 60)

    # Step 3: Add derived columns
    enriched_df = add_derived_columns(cleaned_df)

    print("\n" + "=" * 60)

    # Step 4: Validate data quality
    validate_silver_data(enriched_df)

    print("\n" + "=" * 60)

    # Step 5: Select final columns
    silver_df = select_silver_columns(enriched_df)

    print("\n" + "=" * 60)

    # Step 6: Save Silver data
    save_silver_data(silver_df, silver_path)

    print("\n" + "=" * 60)
    print("🎉 Silver Layer Processing Pipeline Completed Successfully!")

    return silver_df

In [21]:
# Execute the Silver layer processing pipeline
silver_df = process_silver_layer()

# Display sample of final Silver data
if silver_df:
    print("\n📊 Sample of Silver Layer Data:")
    silver_df.show(10, truncate=False)

    print("\n📋 Silver Layer Schema:")
    silver_df.printSchema()

🏗️ Starting Silver Layer Processing Pipeline...
✅ Successfully loaded Bronze data from /content/election_results as CSV
📊 Total records: 8902
📋 Columns: 13

🧹 Starting data cleaning and standardization...
✅ Data cleaning and standardization completed

🔧 Adding derived columns...
✅ Derived columns added successfully

🔍 Performing Silver layer data validation...

=== Data Consistency Checks ===
Records with vote calculation mismatch: 0
Records with negative EVM votes: 0
Records with negative Postal votes: 0
Records with invalid vote percentage: 0

=== Winner Validation ===
Constituencies with multiple winners: 0
Constituencies with no winners: 0

=== Silver Layer Summary ===
Total records: 8902
Total constituencies: 543
Total states: 36
Total unique parties: 746

=== Party Type Distribution ===
+-----------------+-----+
|       Party_Type|count|
+-----------------+-----+
|      Independent| 3920|
|           Others| 2843|
|National/Regional| 1597|
|             NOTA|  542|
+-------------

In [22]:
# Additional analysis and insights
if silver_df:
    print("\n🔍 Additional Silver Layer Insights:")
    print("=" * 50)

    # Top 10 parties by number of candidates
    print("\n🏛️ Top 10 Parties by Number of Candidates:")
    silver_df.groupBy("Party").count() \
        .orderBy(desc("count")) \
        .show(10, truncate=False)

    # Winners by party
    print("\n🏆 Seats Won by Party:")
    silver_df.filter(col("Is_Winner") == True) \
        .groupBy("Party").count() \
        .orderBy(desc("count")) \
        .show(20, truncate=False)

    # State-wise constituency count
    print("\n🗺️ Constituencies per State:")
    silver_df.groupBy("State") \
        .agg(countDistinct("Constituency").alias("Constituency_Count")) \
        .orderBy(desc("Constituency_Count")) \
        .show(10, truncate=False)

    # Average vote percentage by party type
    print("\n📊 Average Vote Percentage by Party Type:")
    silver_df.groupBy("Party_Type") \
        .agg(avg("Vote_Percentage").alias("Avg_Vote_Percentage"),
             count("*").alias("Total_Candidates")) \
        .orderBy(desc("Avg_Vote_Percentage")) \
        .show(truncate=False)


🔍 Additional Silver Layer Insights:

🏛️ Top 10 Parties by Number of Candidates:
+-------------------------------------------+-----+
|Party                                      |count|
+-------------------------------------------+-----+
|Independent                                |3920 |
|None of the Above                          |542  |
|Bahujan Samaj Party                        |488  |
|Bharatiya Janata Party                     |441  |
|Indian National Congress                   |328  |
|Socialist Unity Centre Of India (COMMUNIST)|150  |
|Peoples Party of India (Democratic)        |79   |
|Samajwadi Party                            |71   |
|Communist Party of India (Marxist)         |52   |
|All India Trinamool Congress               |48   |
+-------------------------------------------+-----+
only showing top 10 rows


🏆 Seats Won by Party:
+------------------------------------------------+-----+
|Party                                           |count|
+---------------------------

## Silver Layer Summary

The Silver layer processing has successfully:

### ✅ Data Cleaning & Standardization
- Cleaned and standardized text fields (names, parties, constituencies)
- Handled missing values and special characters in vote counts
- Converted string columns to appropriate data types
- Standardized result status and party classifications

### 🔧 Data Enrichment
- Added derived columns for better analysis
- Calculated constituency rankings and winner identification
- Created party type classifications (National/Regional, Independent, Others)
- Added vote share categories for easy segmentation

### 🔍 Data Validation
- Verified data consistency and integrity
- Validated one winner per constituency rule
- Checked for data quality issues and anomalies
- Provided comprehensive quality reports

### 💾 Output Formats
- Saved as optimized Parquet files for analytics
- Also provided CSV format for easy viewing
- Maintained traceability with Bronze layer metadata

The Silver layer data is now ready for advanced analytics, reporting, and Gold layer aggregations!